In [4]:
##제일먼저 설치해주세요

!pip install xgboost 
!pip install fancyimpute

In [5]:
##사용 라이브러리

import os
import time
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


from xgboost import XGBRegressor
from fancyimpute import IterativeImputer
from sklearn.model_selection import KFold 
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

In [6]:
#seed 고정

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed=42
seed_everything(seed)


In [7]:
#데이터
path = '@@@@@'

train_csv = @@@(path + 'train.csv')
test_csv = @@@(path + 'test.csv')
submit_csv = @@@(path + 'sample_submission.csv')

#.데이터 제대로 붙은지 확인하기
# print(train_csv)
# print(test_csv)
# print(submit_csv)



In [8]:
#찾아보기!#
#iterativeImputer (결측치 처리 방법중 하나)
imputer = IterativeImputer(random_state=42)
train_csv['배터리용량'] = imputer.fit_transform(train_csv[['배터리용량']])
test_csv['배터리용량'] = imputer.transform(test_csv[['배터리용량']]) 

In [9]:
#훈련할 변수
x = @@@
y = @@@

#테스트 파일
x_test1 = @@@

In [ ]:
### @@필요라이브러리 import 

#라벨인코딩
categorical_features = [col for col in x.columns if x[col].dtype == 'object']

# LabelEncoder 적용
for col in categorical_features:
    le = @@@
    x[col] = le.fit_transform(x[col])
    x_test1[col] = le.transform(x_test1[col])


In [ ]:
#이상치처리

# def fit_outlier(data):  
#     data = @@.DataFrame(data) #데이터를 dataframe 형식으로 변환
#     for label in data:
#         series = data[label]
#         q1 = series.quantile(0.25)      
#         q3 = series.quantile(0.75)
#         iqr = q3 - q1
#         upper_bound = q3 + iqr
#         lower_bound = q1 - iqr
        
#         series[series > upper_bound] = np.nan
#         series[series < lower_bound] = np.nan
#         print(#@@결측치 확인@@)
#         series = series.interpolate()
#         data[label] = data[label].interpolate() 
        
#     data = data.fillna(data.@@@)
#     return data

# #이상치처리한 데이터로 바꾸기
# x = @@@@@






In [15]:
# # 스케일링
# scaler = StandardScaler()

# x_scaled = @@@
# x_test1 = @@@@


In [13]:
###교차검증###

# KFold 설정
n_splits = @@
kfold = KFold(n_splits=@@, shuffle=@@, random_state=42)


In [16]:
##파라미터 설정

best_params = {
    'n_estimators': 7000, 
    'max_depth': 6,
    'learning_rate': 0.022967280267686598, 
    'min_child_weight': 1,
    'gamma': 0.01585944217292061,
    'subsample': 0.9583496415625028, 
    'colsample_bytree': 0.9124943873276311,
    'lambda': 9.133524592946834, 
    'alpha': 2.379891161256474}


In [17]:
# 결과 저장
r2_scores =  @@
rmse_scores = @@
models = @@

In [19]:
# KFold 교차 검증
for fold, (train_index, val_index) in enumerate(kfold.split(x_scaled)):
    print(f"\n--- Fold {fold + 1} ---")
    
    # 훈련 및 검증 데이터 분리
    x_train_fold, x_val_fold = x_scaled[train_index], x_scaled[val_index]
    y_train_fold, y_val_fold = y[train_index], y[val_index]
    
    # 모델 생성
    model = XGBRegressor(@@)
    
    
    # 모델 학습
    start_time = time.time()
    model.fit(
        x_train_fold, y_train_fold,
        eval_set=[(x_val_fold, y_val_fold)],  
        
        verbose=False
    )
    end_time = time.time()
    
    # 예측 및 평가
    y_val_pred = model.predict(@@@)
    r2 = r2_score(@@@)
    rmse = np.sqrt(mean_squared_error(@@@))
    
    r2_scores.append(r2)
    rmse_scores.append(rmse)
    models.append(model)
    
    print(f"Fold {fold + 1} R2 Score: {r2:.4f}, RMSE: {rmse:.4f}")
    print(f"Fold {fold + 1} Training Time: {round(end_time - start_time, 2)} seconds")


--- Fold 1 ---
Fold 1 R2 Score: 0.9986, RMSE: 1.3929
Fold 1 Training Time: 3.21 seconds

--- Fold 2 ---
Fold 2 R2 Score: 0.9987, RMSE: 1.3382
Fold 2 Training Time: 3.57 seconds

--- Fold 3 ---
Fold 3 R2 Score: 0.9984, RMSE: 1.4482
Fold 3 Training Time: 3.11 seconds

--- Fold 4 ---
Fold 4 R2 Score: 0.9987, RMSE: 1.3633
Fold 4 Training Time: 3.81 seconds

--- Fold 5 ---
Fold 5 R2 Score: 0.9986, RMSE: 1.3823
Fold 5 Training Time: 3.95 seconds

--- Fold 6 ---
Fold 6 R2 Score: 0.9992, RMSE: 1.0344
Fold 6 Training Time: 3.34 seconds

--- Fold 7 ---
Fold 7 R2 Score: 0.9984, RMSE: 1.3815
Fold 7 Training Time: 3.96 seconds


In [ ]:
# 최종 모델로 전체 데이터 학습
final_model = XGBRegressor(**best_params)
final_model.fit(
    @@@, @@,
    eval_set=[(x_val_fold, y_val_fold)],
    verbose=False
)

# 테스트 데이터 예측
y_submit = final_model.predict(@@@)

# 결과 저장
@@@@ = y_submit
# print(submit_csv)

ltm = time.localtime(time.time())
save_time = f"{ltm.tm_year}{ltm.tm_mon}{ltm.tm_mday}{ltm.tm_hour}{ltm.tm_min}{ltm.tm_sec}"


submit_csv.to_csv(path + f"submission_{save_time}_rmse_{np.mean(rmse_scores):.4f}.csv", index=False)

